In [7]:
import torch
import numpy as np
import pandas as pd
import segmentation_models_pytorch as sm
import cv2
import os
import glob
from tqdm import tqdm
import multiprocessing as mp
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import io
import torchvision.transforms as T

In [8]:
csv_file = pd.read_csv('../../files/train_ship_segmentations_v2.csv')
csv_file = csv_file.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
image_ids, pixels = csv_file['ImageId'].values.tolist(), csv_file['EncodedPixels'].values.tolist()

In [9]:
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../files/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../files/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [10]:
for x in tqdm(csv_file['fixed_inputs'].values.tolist()):
    if os.path.exists(x) == False:
        print(x)

100%|██████████| 192556/192556 [00:00<00:00, 622437.67it/s]


In [11]:
for x in tqdm(csv_file['mask_paths'].values.tolist()):
    if os.path.exists(x) == False:
        print(x)

100%|██████████| 192556/192556 [00:00<00:00, 606689.04it/s]


In [12]:
csv_file['fixed_inputs'].values.tolist()[0]

'../../files/train_v2/00003e153.jpg'

In [13]:
csv_file = csv_file[csv_file['fixed_inputs'] != '../../files/train_v2/6384c3e78.jpg']

In [14]:
def split_datasets(csv_file, test_size = 0.01):
    train, test = train_test_split(csv_file, test_size = test_size, random_state=42)
    train, val = train_test_split(train, test_size = test_size, random_state=42)
    return train, val, test

In [15]:
# csv_file = pd.read_csv
train, val, test = split_datasets(csv_file)

In [16]:
type_of_label = []
for x in train['EncodedPixels'].values.tolist():
    if type(x[0]) == str:
        type_of_label.append(1)
    else:
        type_of_label.append(0)

In [17]:
unet_incep = sm.Unet(
    encoder_name='inceptionv4',
    encoder_weights=None,
    in_channels=3,
    classes=2
)

In [18]:
len(train)

188722

In [19]:
np.random.randint(0, 2, size = (188722))

array([0, 0, 0, ..., 1, 0, 0])

In [20]:
T.functional.hflip

<function torchvision.transforms.functional.hflip(img: torch.Tensor) -> torch.Tensor>

In [21]:
np.random.uniform(1, 2, 5)

array([1.17191695, 1.58495319, 1.21061973, 1.83898429, 1.64009964])

In [22]:
class ShipSegmentationData(Dataset):
    def __init__(self, csv_file, output_shape):
        self.csv_file = csv_file
        self.imgs = self.csv_file['fixed_inputs'].values.tolist()
        self.masks = self.csv_file['mask_paths'].values.tolist()
        self.flip_probs = np.random.randint(0, 2, size = (len(self.imgs)))
        self.brightness_factor = np.random.uniform(1, 2, len(self.imgs))
        self.contrast_factor = np.random.uniform(1, 2, len(self.imgs))
        self.output_shape = output_shape

    def __getitem__(self, idx):
        img = io.read_image(self.imgs[idx])
        mask = io.read_image(self.masks[idx])
        if self.flip_probs[idx] == 1:
            img = T.functional.hflip(img)
            mask = T.functional.hflip(mask)
        img = T.functional.adjust_brightness(img, self.brightness_factor[idx])
        img = T.functional.adjust_contrast(img, self.contrast_factor[idx])
        img = T.functional.resize(img, self.output_shape)
        mask = T.functional.resize(mask, self.output_shape)
        img = img / 255
        mask = mask / 255
        mask = torch.where(mask < 1, 0, 1)
        return img, mask

    def __len__(self):
        return len(self.imgs)

In [23]:
obj = ShipSegmentationData(train, output_shape=(512, 512))
img, mask = obj.__getitem__(6)

/home/ubuntu/torch_env/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
